# Synthesize Responses from the Retrieved Contexts

## Step 0: Environment Setup

In [2]:
from dotenv import load_dotenv
import os
from langchain_neo4j import Neo4jGraph
from libs import create_vector_index
import pandas as pd
# plot figures in high resolution
%config InlineBackend.figure_format = 'retina'
# Force reload of the .env file
try:
    load_dotenv()
    print("The .env file has been loaded")
except Exception as e:
    print("The .env file has not been loaded. Reported error: ", e)

The .env file has been loaded


In [3]:
# Connect to Neo4j database
try:
    graph = Neo4jGraph(
        url=os.getenv("NEO4J_URL"),
        username=os.getenv("NEO4J_USERNAME"),
        password=os.getenv("NEO4J_PASSWORD")
    )
    print("Connected to Neo4j database successfully.")
except ValueError as e:
    print(f"Could not connect to Neo4j database: {e}")
    
# Check if the entities index exists
index_name = "entities"
query = "SHOW INDEXES YIELD name, type WHERE type = 'VECTOR' AND name = $index_name"

result = graph.query(query, params={"index_name": index_name})
if result:
    print("The 'entities' index already exists.")
else:
    create_vector_index(graph, "entities")

Connected to Neo4j database successfully.
The 'entities' index already exists.


## Step 1: Connect to Google Sheet

In [ ]:
spreadsheet = connect2Googlesheet()

# Select the worksheet: relevance
Q_worksheet = spreadsheet.get_worksheet(2)  
# Select the worksheet: revised summary
Sum_worksheet = spreadsheet.get_worksheet(6) 

# Get all records as a list of dictionaries
ques_data = Q_worksheet.get_all_records()
sum_data = Sum_worksheet.get_all_records()

# Convert to Pandas DataFrame
df_MedQ = pd.DataFrame(ques_data)
df_MedSum = pd.DataFrame(sum_data)

# Display the first few rows of both DataFrames
display(df_MedQ.head())
display(df_MedSum.head())

,condition,number,docs,Question,Mahmud's Note,status,comments,
0,ARDS,1,ACURASYS,Does early administration of neuromuscular blo...,Like,,,
1,ARDS,2,ACURASYS,Do patients with severe ARDS being treated wit...,Replace,fixed,,
2,ARDS,3,ROSE,"In patients with moderate to severe ARDS, does...",Maybe this question: In patients with moderate...,fixed,,
3,ARDS,4,ROSE,Do patients with moderate-to-severe ARDS have ...,Local question (not sure if this is the aim of...,fixed,Wrong concept since PEEP by itself is mandator...,Does the use of neuromuscular blockers in pati...
4,ARDS,5,FACTT,"Among patients with ALI/ARDS, does a conservat...",Local question (not sure if this is the aim of...,fixed,Check if studies defined conservative by CVP <...,


,Doc No,Confirmed by,Topic,Name,Study Design and Methodology,Study Population,Interventions,Comparator,Outcomes,Strengths and Weaknesses,Key Findings and Conclusion
0,1,,ARDS,ACURASYS,"- Multicenter, randomized, double-blind, place...",The study enrolled 340 patients from 20 ICUs i...,- Cisatracurium Group:\n - 3-ml rapid IV infu...,- Placebo group receiving identical IV infusio...,Primary Outcome:\n- 90-day in-hospital mortali...,Strengths:\n- Multicenter design\n- Blinded ra...,Key Findings:\n- Adjusted 90-day survival impr...
1,6,,ARDS,ROSE,"- Multicenter, unblinded, randomized clinical ...",- Patients with moderate-to-severe ARDS\n- Cri...,- 48-hour continuous infusion of cisatracurium...,- Usual care approach\n- No routine neuromuscu...,Primary Outcome:\n- In-hospital death from any...,Strengths:\n- Multicenter design\n- High proto...,Key Findings:\n No significant difference in 9...
2,3,,ARDS,FACTT,"- Prospective, randomized clinical trial\n- Co...","- 1,001 patients with acute lung injury (503 i...",Conservative Strategy:\n- Lower fluid intake\n...,The two fluid management strategies were compa...,Primary Outcome:\n- Death at 60 days\n\nSecond...,Strengths:\n- Randomized design\n- Multi-cente...,Key Findings:\n- No significant difference in ...
3,2,,ARDS,ARDSNet,"- Multicenter, randomized controlled trial\n- ...",- Total Participants: 861 patients\n- Inclusio...,- Lower Tidal Volume Group:\n - Initial tidal...,- Traditional Tidal Volume Group:\n - Initial...,Primary Outcomes:\n- Mortality before hospital...,Strengths:\n- Multicenter design\n- Robust ran...,Key Findings:\n- Mortality reduced by 22% in l...
4,5,,ARDS,PROSEVA,"- Multicenter, prospective, randomized, contro...",- 466 patients with severe ARDS\n- Inclusion c...,- Prone group (237 patients):\n - Turned to p...,- Supine group (229 patients):\n - Remained i...,Primary Outcome:\n- 28-day mortality\n\nSecond...,Strengths:\n- Multicenter design\n- Standardiz...,Key Findings:\n- 28-day mortality: 16.0% (pron...


## Step 2: Build Context before Synthesis